<a href="https://colab.research.google.com/github/LaurentiaNaidu/FYP/blob/main/transfer_learning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]


Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In [2]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

58900480/58889256 [==============================] - 1s 0us/step


In [3]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 12s 170ms/step - loss: 0.1684 - accuracy: 0.9440 - val_loss: 1.5159 - val_accuracy: 0.6868
Epoch 2/50
65/65 [==============================] - 11s 164ms/step - loss: 0.1769 - accuracy: 0.9445 - val_loss: 2.0838 - val_accuracy: 0.6848
Epoch 3/50
65/65 [==============================] - 11s 164ms/step - loss: 0.1325 - accuracy: 0.9591 - val_loss: 1.7660 - val_accuracy: 0.6693
Epoch 4/50
65/65 [==============================] - 11s 164ms/step - loss: 0.0634 - accuracy: 0.9796 - val_loss: 1.7163 - val_accuracy: 0.7004
Epoch 5/50
65/65 [==============================] - 11s 163ms/step - loss: 0.0231 - accuracy: 0.9942 - val_loss: 1.6829 - val_accuracy: 0.7160
Epoch 6/50
65/65 [==============================] - 11s 165ms/step - loss: 0.0161 - accuracy: 0.9961 - val_loss: 1.7787 - val_accuracy: 0.7023
Epoch 7/50
65/65 [==============================] - 11s 163ms/step - loss: 0.0083 - accuracy: 0.9995 - val_loss: 1.8284 - val_accuracy: 0.7296